# 3. 벡터와 행렬 연산 (II)

## 3.1. 행렬-행렬 곱

1. 순차코드

    <img src = "images/image08.png">

In [ ]:
import numpy as np

A = np.load("examples/A.npy")
B = np.load("examples/B.npy")

C = np.matmul(A, B)
print (C)


2. 행렬A의 행분할

    <img src = "images/image09.png">

In [ ]:
%%writefile examples/AB.py

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0 :
    A = np.load("examples/A.npy")
    B = np.load("examples/B.npy")
    n = A[0].size
    n = comm.bcast(n, root = 0)

else :
    n = 0
    n = comm.bcast(n, root = 0)
    A = None
    B = np.empty((n, n), dtype = np.float64)

n_local = int(n / size)

A_local = np.empty((n_local, n), dtype = np.float64)

##### 행렬 A의 분할
comm.Scatter(A, A_local, root = 0) #FIX ME
comm.Bcast(B, root = 0)

C = np.matmul(A_local,B)

print(C, rank)

In [ ]:
! mpiexec -np 2 python examples/AB.py

3. 행렬A의 행분할 (비등분할)

    <img src = "images/image10.png">

In [ ]:
%%writefile examples/AvarB.py

# Matrix A의 Row decomposition

import numpy as np
from mpi4py import MPI
from tools import para_range

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

##### 행렬 및 벡터 불러오기
##### 크기 n은 broadcast
if rank == 0 :
    A = np.load("examples/A.npy")
    B = np.load("examples/B.npy")
    n = A[0].size
    n = comm.bcast(n, root = 0)
else :
    n = 0
    n = comm.bcast(n, root = 0)
    A = None
    B = np.empty((n, n), dtype = np.float64)

##### 행렬 A의 분할 범위를 설정
ista, iend = #FIX ME
n_local = #FIX ME

##### Scatterv를 위해 n_local를 이용한 리스트 생성. 이 때 부분행렬 행수가 아닌 전체 크기를 계산 
A_local = np.empty((n_local, n), dtype = np.float64)
n_local_chunks = comm.gather(n_local * n, root = 0)

##### 행렬 A의 분할
comm.Scatterv( #FIX ME
comm.Bcast(B, root = 0)

##### 분할된 부분행렬과 행렬B의 곱
C_local = np.matmul(A_local,B)

print(C_local, rank)

In [ ]:
! mpiexec -np 3 python examples/AvarB.py

4. 행렬A의 행분할, 행렬 B의 열분할
  - 각 프로세스가 가진 부분행렬만 곱할 경우

    <img src = "images/image11_1.png">

  - 통신을 이용하여 $B_i$에 대한 부분행렬을 수신하여 부분행렬 곱을 실행

    <img src = "images/image11_2.png">


In [ ]:
%%writefile examples/ABvar.py

import numpy as np
from mpi4py import MPI
from tools import para_range

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

##### 행렬 및 벡터 불러오기
##### 크기 n은 broadcast
if rank == 0 :
    A = np.load("examples/A.npy")
    B = np.load("examples/B.npy")
    BT = np.transpose(B).copy()
    n = A[0].size
    n = comm.bcast(n, root = 0)

else :
    n = 0
    n = comm.bcast(n, root = 0)
    A = None
    BT = None

##### 행렬 A와 B의 분할 범위를 설정
ista, iend = para_range(n, size, rank)
n_local = (iend - ista + 1)

##### 부분행렬의 선언. 행렬 B의 부분행렬은 전치행렬을 이용
A_local = np.empty((n_local, n), dtype = np.float64)
BT_local = np.empty((n_local, n), dtype = np.float64)

##### Scatterv를 위해 n_local를 이용한 리스트 생성. 이 때 부분행렬 행수가 아닌 전체 크기를 계산 
n_local_chunks = comm.allgather(n_local * n)

##### 행렬 A, B의 분할
comm.Scatterv( #FIX ME
comm.Scatterv( #FIX ME

##### 행렬 B의 부분행렬을 전치하여 원래 형태로 만듦
B_col = np.transpose(BT_local)

##### 다음 및 이전 랭크 설정
inext = rank + 1 if rank < size - 1 else 0
iprev = rank - 1 if rank > 0 else size - 1

##### 계산결과는 정렬되지 않으므로 unordered로 취급
C_unordered_local = np.matmul( #FIX ME


##### (전체 프로세스 크기 -1) 번만큼 통신을 수행하고 부분행렬-부분벡터 곱을 수행
for i in range(size - 1) :
##### 몇 번째 이전 랭크로부터 받았는지를 iloc을 이용하여 확인
    iloc = iprev - i if iprev >= i else iprev - i + size
    B_recv = np.empty(n_local_chunks[iloc], dtype = np.float64)

    ##### 분할된 행렬 B를 송수신하고 A의 분할된 부분과 곱하여 C에 저장
    comm.Sendrecv( #FIX ME
    B_col = np.copy(B_recv)
    B_col = np.reshape(B_col, (n, int(n_local_chunks[iloc]/n)))
    C_block = np.matmul( #FIX ME
##### 현재 정렬되지 않은 순서로 C_unordered_local_chunks에 저장됨    
    C_unordered_local = np.append(C_unordered_local, C_block, axis = 1)

print(C_unordered_local, rank)

In [ ]:
! mpiexec -np 3 python examples/ABvar.py

5. 직접 해보기
    - 올바른 크기의 C를 미리 선언하고 적절한 위치에 C_block 을 배치
    - 3의 C_rows와 같은 결과를 얻음
